In [1]:
import pandas as pd
import numpy as np
import pyodbc
import math
from ast import literal_eval
from tqdm import tqdm
from configparser import ConfigParser

# from connector.mosaicio import MosaicioConnector

In [2]:
pd.set_option("display.max_column",200)
pd.set_option("display.float_format",lambda x:'%.5f' %x)

In [3]:
pwd

'/notebooks/notebooks'

In [4]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [5]:
cursor.execute('''
merge into FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_ANALYTICAL_RULES as target using( 
select A.ORGANIZATION_MASTER_ID,A.RULE_NO,
coalesce((case    
when c.THRESHOLD1_1 IS null then 1 
when A.RULE_VALUE < c.threshold1_1 then 0 
when A.RULE_VALUE = c.threshold1_1 then 1 
when A.RULE_VALUE <= c.THRESHOLD1_2 then 2  
when A.RULE_VALUE <= c.THRESHOLD1_3 then 3 
when A.RULE_VALUE <= c.THRESHOLD1_4 then 4 
when A.RULE_VALUE <= c.THRESHOLD1_5 then 5 
when A.RULE_VALUE <= c.THRESHOLD1_6 then 6 
when A.RULE_VALUE <= c.THRESHOLD1_7 then 7 
when A.RULE_VALUE <= c.THRESHOLD1_8 then 8 
when A.RULE_VALUE <= c.THRESHOLD1_9 then 9 
when A.RULE_VALUE <= c.THRESHOLD1_10 then 10 
else 10    end),0) Risk_score, 
GETDATE() AS CREATE_DATE from FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG_rules a
inner join FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG b 
on a.ORGANIZATION_MASTER_ID = b.ORGANIZATION_MASTER_ID inner join 
FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_INCREMENTAL d 
on b.ORGANIZATION_MASTER_ID = d.ORGANIZATION_MASTER_ID 
left join FINCORE_DB.Fincore_Analytics.RISK_lkp_entity_threshold c
on c.PEER_GROUP_ID = b.PEER_GROUP_ID and a.RULE_NO =c.RULE_NO 
inner join FINCORE_DB.Fincore_Analytics.risk_lkp_rules rlr with (nolock)
on rlr.rule_no = a.rule_no
where rlr.RISK_SCORE_STATIC_flag = 0) 
as source on target.ORGANIZATION_master_id=source.ORGANIZATION_master_id and 
target.rule_no=source.rule_no 
when matched then update set target.ORGANIZATION_master_id=source.ORGANIZATION_master_id,target.rule_no=source.rule_no,
target.threshold_flg=source.risk_score
when not matched by target then insert(ORGANIZATION_master_id,rule_no,threshold_flg,create_date)
values(source.ORGANIZATION_master_id,source.rule_no,source.risk_score,getdate());''')
conn.commit()

In [44]:

WEIGHTED_AVG = pd.read_sql_query('''

SELECT ORGANIZATION_MASTER_ID,
PRIORITY_NO,
CASE WHEN SUM(THRESHOLD_FLG)!= 0 THEN SUM(SQUARE(THRESHOLD_FLG))/SUM(THRESHOLD_FLG) 
ELSE 0
END AS FINAL
FROM (
SELECT T.*,PRIORITY_NO FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES RLR WITH (NOLOCK)
INNER JOIN 
(

SELECT DISTINCT RPSSR.ORGANIZATION_MASTER_ID,RULE_NO,RISK_SCORE_STATIC AS THRESHOLD_FLG 
FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES RPSSR WITH(NOLOCK)
INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_INCREMENTAL K WITH(NOLOCK) ON K.ORGANIZATION_MASTER_ID = RPSSR.ORGANIZATION_MASTER_ID
WHERE RISK_SCORE_STATIC IS NOT NULL

UNION 

SELECT DISTINCT
A.ORGANIZATION_MASTER_ID,
A.RULE_NO,
coalesce((CASE 
	WHEN C.THRESHOLD1_1 IS NULL THEN 0
	WHEN A.RULE_VALUE < C.THRESHOLD1_1 THEN 0 
	WHEN A.RULE_VALUE = C.THRESHOLD1_1 THEN 1
	WHEN A.RULE_VALUE <= C.THRESHOLD1_2 THEN 2
	WHEN A.RULE_VALUE <= C.THRESHOLD1_3 THEN 3
	WHEN A.RULE_VALUE <= C.THRESHOLD1_4 THEN 4
	WHEN A.RULE_VALUE <= C.THRESHOLD1_5 THEN 5
	WHEN A.RULE_VALUE <= C.THRESHOLD1_6 THEN 6
	WHEN A.RULE_VALUE <= C.THRESHOLD1_7 THEN 7
	WHEN A.RULE_VALUE <= C.THRESHOLD1_8 THEN 8
	WHEN A.RULE_VALUE <= C.THRESHOLD1_9 THEN 9
	WHEN A.RULE_VALUE <= C.THRESHOLD1_10 THEN 10
	ELSE 10
	END),1) THRESHOLD_FLG
	FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES A
INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_INCREMENTAL K WITH(NOLOCK) ON K.ORGANIZATION_MASTER_ID = A.ORGANIZATION_MASTER_ID
INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG B ON A.ORGANIZATION_MASTER_ID = B.ORGANIZATION_MASTER_ID
INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_ENTITY_THRESHOLD C ON C.PEER_GROUP_ID = B.PEER_GROUP_ID AND A.RULE_NO = C.RULE_NO
INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES D ON D.RULE_NO = A.RULE_NO

WHERE D.RISK_SCORE_STATIC_flag = 0
)T
ON T.RULE_NO = RLR.RULE_NO

)T
GROUP BY ORGANIZATION_MASTER_ID,PRIORITY_NO

''',conn)


In [45]:
weights = pd.read_sql_query('''Select distinct PRIORITY_NO, WEIGHTAGE as WEIGHTS FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES WITH (NOLOCK)''',conn)
weights.dropna(inplace = True)

In [46]:
WEIGHTED_AVG = WEIGHTED_AVG.merge(weights,how = 'inner',on = 'PRIORITY_NO')
WEIGHTED_AVG['WEIGHTED_SCORE'] = WEIGHTED_AVG['FINAL']*WEIGHTED_AVG['WEIGHTS']
WEIGHTED_AVG.drop(columns = ['PRIORITY_NO','FINAL','WEIGHTS'],inplace = True)
FINAL_DF = WEIGHTED_AVG.groupby(['ORGANIZATION_MASTER_ID']).sum()
FINAL_DF.reset_index(inplace = True)

In [47]:
risk_ORGANIZATION = pd.read_sql_query('''SELECT RPSS.ORGANIZATION_MASTER_ID,PEER_GROUP_ID FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG RPSS WITH(NOLOCK)
INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_INCREMENTAL K WITH(NOLOCK) 
ON RPSS.ORGANIZATION_MASTER_ID = K.ORGANIZATION_MASTER_ID ''',conn)


In [48]:
auto_high_df = pd.read_sql_query('''SELECT RPSSR.ORGANIZATION_MASTER_ID FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES RPSSR WITH(NOLOCK)
INNER JOIN FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_INCREMENTAL K WITH(NOLOCK) ON K.ORGANIZATION_MASTER_ID = RPSSR.ORGANIZATION_MASTER_ID
INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES RLR WITH (NOLOCK) ON RLR.RULE_NO = RPSSR.RULE_NO 
WHERE RISK_TYPE = 'AUTO HIGH RULES '
''',conn)

In [49]:
auto_high_ORGANIZATION = auto_high_df['ORGANIZATION_MASTER_ID'].to_list()

In [50]:
FINAL_DF = FINAL_DF[~FINAL_DF['ORGANIZATION_MASTER_ID'].isin(auto_high_ORGANIZATION)]

In [51]:
FINAL_DF = FINAL_DF.merge(risk_ORGANIZATION,how = 'inner', on = 'ORGANIZATION_MASTER_ID')

In [52]:
FINAL_DF.rename(columns = {'WEIGHTED_SCORE':'RISK_SCORE'},inplace = True)

In [53]:
FINAL_DF

PERSON_MASTER_ID  RISK_SCORE        PEER_GROUP_ID
0                   1     6.08155  Government Sector_0
1                   2     5.17500  Government Sector_0
2                   3     5.17500  Government Sector_0
3                   4     5.17500  Government Sector_0
4                   5     5.17500  Government Sector_0
..                ...         ...                  ...
416               532     0.00000             Others_0
417               533     0.00000             Others_0
418               534     7.23846     Private Sector_0
419               535     6.83720     Private Sector_0
420               536     7.84211     OTHERS_CHANGED_9

[421 rows x 3 columns]

In [54]:
FINAL_DF['RISK_SCORE'] = FINAL_DF['RISK_SCORE'].apply(lambda x: round(x))

In [55]:
FINAL_DF['RISK_SCORE'] = FINAL_DF['RISK_SCORE'].astype('int64')

In [56]:
count = 1
for ind,row in tqdm(FINAL_DF.iterrows()):
    if count%10000 == 0:
        print(count,' HERE')
        conn.commit()
    count += 1
    try:
        cursor.execute('''insert into FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION (ORGANIZATION_MASTER_ID,PEER_GROUP_ID,RISK_SCORE,CREATE_DATE)
    values ({},'{}',{},{})'''.format(row['ORGANIZATION_MASTER_ID'],row['PEER_GROUP_ID'],row['RISK_SCORE'],'Getdate()'))
        conn.commit()
    except:
        cursor.execute('''update FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION set peer_group_id = '{}' ,
        risk_score = '{}' , create_date = getdate() where ORGANIZATION_master_id = {}  '''.format(row['PEER_GROUP_ID'],row['RISK_SCORE'],row['ORGANIZATION_MASTER_ID']))
        
        conn.commit()

    
    
conn.commit()






421it [00:12, 33.57it/s]


In [58]:
cursor.execute('''Update fincore_db.fincore_analytics.risk_ORGANIZATION set risk_score = 1 where risk_score = 0''')
conn.commit()

ProgrammingError: The cursor's connection has been closed.

In [ ]:
%store -r

In [ ]:
cursor.execute(f'''Update FINCORE_DB.Fincore_Analytics.AUDIT_INTGN set JOB_END_DATE = Getdate(),JOB_STATUS = 'SUCCESS',NO_OF_RECORD = {no_of_record} where  JOB_RUN_ID ={job_id}''')

cursor.commit()
conn.close()

In [ ]:
conn.close()

In [161]:
print("Entity_Risk_Calculation_V2_1_Incremental_data completed")

Entity_Risk_Calculation_V2_1_Incremental_data completed
